### The script is used to download planetscope images

* Planet has updated their SDK to version 2 in May 2023. The original code used to download planetscope images is not working. Below is a modified version (Kehan Yang, 2023-05-26)

In [2]:
from planet import OrdersClient, Session

async with Session() as session:
    client = OrdersClient(session)
    result = await client.create_order(order)
# Process result


NameError: name 'order' is not defined

### Authenticate with Planet Services

The SDK provides the auth.Auth class for managing authentication information. This module can be used to obtain authentication information from the username and password with Auth.from_login(). Additionally, it can be created with the API key obtained directly from the Planet account site with Auth.from_key(<API_KEY>).

Once you have provided the authentication information (in other words, the username and API key), it can be accessed by way of the Auth.value. The most convenient way of managing it for local use is to write it to a secret file using Auth.write(). For example, to obtain and store authentication information:

Once you have provided the authentication information (in other words, the account username and password), it can be accessed by way of Auth.value. The most convenient way of managing it for local use is to write it to a secret file using Auth.write(). It can also be accessed, for example, to store in an environment variable, such as Auth.value.

Here is an example of retrieving and storing authentication information:

In [ ]:
# Get the user account name and password
# from the command line and environment,
# and store credentials in an Auth object
import getpass
from planet import Auth

user = input("Username: ")
pw = getpass.getpass()
auth = Auth.from_login(user,pw)
auth.store() 


### Create an order request¶

As a first step in ordering, you create an order request object. This request object is transmitted to the Planet service as a JSON object. The SDK provides a way for you to build up that object: planet.order_request.build_request(). The following code returns an order request object, with the values you’ve provided for:

* a name for your order
* what product to order—in this example, PSScene items with analytic_udm2 product bundle asset types
* what tools to use—here, the clip tool with the area of interest (AOI) to clip within

In [9]:
def create_request():
    # This is your area of interest for the Orders clipping tool
    oregon_aoi = {
       "type":
       "Polygon",
       "coordinates": [[[-117.558734, 45.229745], [-117.452447, 45.229745],
                        [-117.452447, 45.301865], [-117.558734, 45.301865],
                        [-117.558734, 45.229745]]]
   }

   # In practice, you will use a Data API search to find items, but
   # for this example take them as given.
    oregon_items = ['20200909_182525_1014', '20200909_182524_1014']
    oregon_order = planet.order_request.build_request(
       name='oregon_order',
       products=[
           planet.order_request.product(item_ids=oregon_items,
                                        product_bundle='analytic_udm2',
                                        item_type='PSScene')
       ],
       tools=[planet.order_request.clip_tool(aoi=oregon_aoi)])
    
    return oregon_order


Once the order request is built, create an order within the context of a Session with the OrdersClient create_order() function and pass the order request object in:

In [10]:
async def main():
    # Create a session and client
    # The Orders API client is also a subclass of the Session
    # class, so it has all the same methods.
    async with planet.Session() as sess:

        # 'orders' is the service name for the Orders API.
        cl = sess.client('orders')

        request = create_request()
        
        order = await cl.create_order(request)


### Waiting and downloading an order¶

After creating an order client, there is typically a waiting period before the assets can be downloaded. During this time, the order is being processed and customized according to the specifications provided in the order request. To monitor the progress of the order creation process and determine when the assets are ready for download, you can use the wait method provided by the Orders API client.

With wait and download, it is often desired to track progress as these processes can take a long time. To track the progress of the order, the following example code uses a progress bar from the reporting module to report the wait status. The download_order method has built-in reporting capabilities, so we don’t need to use a progress bar for the download process.




In [11]:
from planet import reporting

async def create_wait_and_download():
    async with Session() as sess:
        cl = sess.client('orders')
        with reporting.StateBar(state='creating') as bar:
            # create order
            order = await cl.create_order(request)
            bar.update(state='created', order_id=order['id'])

            # poll
            await cl.wait(order['id'], callback=bar.update_state)

        # download
        await cl.download_order(order['id'])

asyncio.run(create_poll_and_download())


NameError: name 'create_poll_and_download' is not defined

In [ ]:
from datetime import datetime
from planet import data_filter
sfilter = data_filter.and_filter([
    data_filter.permission_filter(),
    data_filter.date_range_filter('acquired', gte=datetime(2022, 6, 1), lte = datetime(2022, 6, 3)),
    data_filter.
])



async def create_and_download(client, order_detail, directory):
    with planet.reporting.StateBar(state='creating') as reporter:
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])
        await client.wait(order['id'], callback=reporter.update_state)

    await client.download_order(order['id'], directory, progress_bar=True)

async def main():
    async with planet.Session() as sess:
        cl = sess.client('orders')
        items = [i async for i in cl.search(['PSScene'], sfilter)]

        # Create the order request
        request = create_request()

        # Create and download the order
        order = await create_and_download(cl, request, DOWNLOAD_DIR)


### Validating checksums¶

Checksum validation provides for verification that the files in an order have been downloaded successfully and are not missing, corrupted, or changed. This functionality is included in the OrderClient, but does not require an instance of the class to be used.

To perform checksum validation:

In [ ]:
from pathlib import Path

# path includes order id
order_path = Path('193e5bd1-dedc-4c65-a539-6bc70e55d928')
OrdersClient.validate_checksum(order_path, 'md5')


In [12]:
import os

# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

In [13]:
# Stockton, CA bounding box (created via geojson.io) 
geojson_geometry = {
  "type": "Polygon",
  "coordinates": [
    [ 
      [-121.59290313720705, 37.93444993515032],
      [-121.27017974853516, 37.93444993515032],
      [-121.27017974853516, 38.065932950547484],
      [-121.59290313720705, 38.065932950547484],
      [-121.59290313720705, 37.93444993515032]
    ]
  ]
}


In [14]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-08-31T00:00:00.000Z",
    "lte": "2016-09-01T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [17]:
import json
import requests
from requests.auth import HTTPBasicAuth

item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

geojson = search_result.json()

# let's look at the first result
print(list(geojson.items()))

[('message', 'Please enter a valid API key.'), ('errors', [])]
